In [1]:
# import pandas as pd
# import torch.utils.data

In [2]:
# Importing lemoncake package - to be able to import code that does not need changing
import sys
sys.path.append('../') 

In [3]:
from lemoncake.data import get_datasets, get_dataloaders

In [4]:
train_ds, val_ds, test_ds = get_datasets()

In [5]:
train_ds.label_counts

{'Atelectasis': 14336.0,
 'Cardiomegaly': 15279.0,
 'Consolidation': 3558.0,
 'Edema': 10310.0,
 'Enlarged Cardiomediastinum': 2309.0,
 'Fracture': 508.0,
 'Lung Lesion': 860.0,
 'Lung Opacity': 13235.0,
 'No Finding': 5013.0,
 'Pleural Effusion': 20322.0,
 'Pleural Other': 305.0,
 'Pneumonia': 3730.0,
 'Pneumothorax': 2902.0}

In [6]:
len(train_ds)

43738

In [8]:
train_ds.get_pos_weights(), val_ds.get_pos_weights(), test_ds.get_pos_weights()

(tensor([  2.,   2.,  11.,   3.,  18.,  85.,  50.,   2.,   8.,   1., 142.,  11.,
          14.]),
 tensor([ 3.,  2., 20.,  3., 22., inf, 39.,  2.,  8.,  2., 52.,  9., 10.]),
 tensor([ 1.,  2., 16.,  2., 14., 75., 46.,  2., 13.,  1., 82., 10., 19.]))

In [23]:
dls = get_dataloaders({'train': train_ds, 'valid': val_ds, 'test': test_ds}, batch_size=32)

In [24]:
train_dl, valid_dl, test_dl = dls['train'], dls['valid'], dls['test']

In [25]:
# batch = next(iter(train_dl))
# x, y = batch['x'], batch['y']
# x.shape, y.shape

# Lightning Models 

In [26]:
from lemoncake.model import *
from pytorch_lightning import Trainer, seed_everything

In [27]:
model = MultimodalBERT(
    train_ds.get_pos_weights(),
    val_ds.get_pos_weights(),
    hidden=384,
    n_layers=6,
    attn_heads=6,
)

In [28]:
seed_everything(42, workers=True)
trainer = Trainer(max_epochs=10, precision='16-mixed')

Global seed set to 42
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [29]:
trainer.fit(model, train_dl, valid_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                | Params
------------------------------------------------------
0 | train_loss_fn | BCEWithLogitsLoss   | 0     
1 | valid_loss_fn | BCEWithLogitsLoss   | 0     
2 | preprocessor  | VectorPreProcessor  | 397 M 
3 | bert          | BERT                | 10.6 M
4 | predictor     | MultiLabelPredictor | 5.0 K 
------------------------------------------------------
407 M     Trainable params
0         Non-trainable params
407 M     Total params
1,631.986 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


# Open Issues / Questions

1. train, valid, test split
2. What to do about "-1" in labels
    - currently doing `y = y.fillna(0).replace(-1, 0)`
    - i.e. replacing `NaN` and `-1` with zeros
    - According to the HAIM paper - `-1` is not determined and they have eliminated everything other than 1 and 0 in their training.
3. Model size options